In [ ]:
#hide
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')


if iskaggle:
    %pip install -Uqq fastai ipywidgets gradio


In [ ]:
#hide
from fastai.vision.all import *
import pandas as pd


# Loading the data

The fastai library provides a URLs module with a pre-specified URL for the flower dataset. We will download and extract the dataset using this module and confirm the contents of the folder by running the "ls" command. The folder should show a "jpg" folder along with "test.txt", "valid.txt" and "train.txt" files.

In [ ]:
path = untar_data(URLs.FLOWERS)

path.ls()

I have used a CSV file mapping label to a specific number,facilitating convenient programming structure.

In [ ]:
flower_df = pd.read_csv('/kaggle/input/oxford_flower_102_name.csv', header=0)

flower_index_dict = flower_df.set_index('Index').to_dict()['Name']
print(flower_index_dict)

To load the `train.txt` file, we are using the pandas.read_csv function. This reads the file name and corresponding index, which is then loaded to a dataframe. We are adding an "is_valid" column to the dataframe to indicate whether each file will be used for training or validation process. Using the dictionary for flower name, we are adding "label" column.

In [ ]:

df = pd.read_csv(f'{path}/train.txt',sep=' ',names = ['fname','index'])
df['is_valid'] = False
df['label'] = df['index'].map(flower_index_dict)

print(df.shape)

df.head()

Using the same steps, we are now loading the `valid.txt` file for validation data. The key difference is that the "is_valid" column has been assigned a value of "True" for each file in this dataframe, indicating that these files will be used for the validation step.

In [ ]:
df_v = pd.read_csv(f'{path}/valid.txt',sep=' ',names = ['fname','index'])
df_v['is_valid'] = True
df_v['label'] = df_v['index'].map(flower_index_dict)
print(df_v.shape)

df_v.head()

To create a single dataframe, we combine the two previously created dataframes using the "concat" command as DataBlock expects a single datasource. We also modified the file names to contain the full path to the image files.

In [ ]:
df = pd.concat([df,df_v])
df.fname = df.fname.apply(lambda x:f'{path}/{x}')
df.head()

# Image Normaliztion

In order to standardize the images, we are defining a transformation pipeline for CPU processing using the fastai library's RandomResizedCrop class. This transformation resizes each image to a random size of 500 pixels while maintaining the aspect ratio. The min_scale parameter specifies the minimum size of the resized image in relation to the original image. The ratio parameter controls the aspect ratio of the resized image, where (1., 1.) indicates a square aspect ratio.

By applying this transformation, the images are in a consistent size while preserving the aspect ratio and minimizing the loss of information.

In [ ]:
item_tfms = [RandomResizedCrop(500,min_scale=0.75, ratio=(1.,1.))]

At this step, we are defining a batch transformation pipeline using the fastai library's aug_transforms and Normalize classes suitable for GPU setting.

The aug_transforms function applies a set of image augmentations, such as flipping, rotating, and zooming, to the images in our dataset to add variations to the images. The size parameter specifies the desired size of the transformed images, which in this case is 244 pixels. I chose to augment because the number of images per category is not too high so augmentation will add diversity for the model.

In the Normalize, the from_stats method is used to specify the normalization statistics, which in this case are the mean and standard deviation values from the ImageNet dataset.


In [ ]:
batch_tfms = [*aug_transforms(size=244),Normalize.from_stats(*imagenet_stats)]

# Creating datablock

It's time to build the datablock to train and validate the model. 
* (ImageBlock, CategoryBlock) specifies the types of data blocks to use for our input images and output categories
* get_x=ColReader('fname') and get_y=ColReader('index') are used to specify the input and output data columns.
* item_tfms=item_tfms and batch_tfms=batch_tfms specify the transformation pipelines to apply to our input data items and batches
* splitter=ColSplitter() is used to specify how to split our dataset into training and validation sets. It will use the column 'is_valid' we defined earlier.


In [ ]:
dblock = DataBlock((ImageBlock,CategoryBlock),
                   get_x=ColReader('fname'),
                   get_y = ColReader('label'),
                   item_tfms = item_tfms,
                   batch_tfms= batch_tfms,
                   splitter=ColSplitter()                  
                  )

Now, the datablock is defined. We can get hold of dataloader object by applying the transformations defined in the 'DataBlock' object to our input dataframe, `df`. 
For visual inspection, `dls.show_batch()` is then used to display a batch of the transformed data. 

In [ ]:
dls = dblock.dataloaders(df)
dls.show_batch()

# Learning with Resnet50

We used the pretrained model, `resnet50` to train the model using the dataloader created before. I chose error rate as a `metrics` parameter to measure the performance of each run. 


In [ ]:
learn_resnet50 = vision_learner(dls, resnet50, metrics=error_rate)
learn_resnet50.fine_tune(6)


fine_tune function allows us to utilize inbuilt Gradient Descent algorithm to finetune the model to Oxford 102 Flower Dataset

In [ ]:
interp_res50 = ClassificationInterpretation.from_learner(learn_resnet50)
interp_res50.plot_confusion_matrix(figsize=(10,10))

We can list some top losses (5 in this case) to understand what it went wrong. 

In [ ]:
interp_res50.plot_top_losses(5, nrows=1)

In [ ]:

learn_resnet50.export(os.path.join('/kaggle/working', 'export.pkl'))


In order to use the exported model file, `load_learner` is called to create an inference object. The inference object also has the access to label vocab to show the prediction value.

In [ ]:
learn_inf = load_learner('/kaggle/working/export.pkl')
learn_inf.dls.vocab

Here is the content for app.py that defines the interface for the UI and service api.

In [ ]:
from fastai.vision.all import * 
import gradio as gr
from PIL import Image

learn_inf = load_learner('/kaggle/working/export.pkl')
labels = learn_inf.dls.vocab
def predict(img):
    img = Image.fromarray(img)
    img = img.resize((512, 512))
    pred,pred_idx,probs = learn_inf.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

title = "Flower Classifier"
description = "Flower classifier for 102 types of flowers"
article="<p style='text-align: center'><a href='https://imju.me' target='_blank'>Blog post</a></p>"
interpretation='default'
enable_queue=True

gr.Interface(
    fn=predict, 
    inputs=gr.Image(type="numpy"), 
    outputs=gr.Label(num_top_classes=3)
).launch(inbrowser=False, share=False)